
**Expanding the Capabilities of Language Models**

In the realm of language models, we are witnessing a revolution marked by their ever-increasing fluency and sophistication. However, the journey doesn't end here. The true challenge lies in harnessing this fluency to develop language models that are not just eloquent, but also reliably capable assistants. This chapter delves into various strategies designed to imbue language models with enhanced intelligence, productivity, and trustworthiness.

The overarching theme of our approach revolves around three core enhancements:

1. **Utilizing Prompts Effectively**: We'll explore how carefully crafted prompts can significantly elevate the performance of language models. This isn't just about asking the right questions; it's about framing these questions in a way that guides the model towards more accurate and relevant responses.

2. **Tool Integration**: By integrating external tools, we can compensate for the inherent limitations of language models, particularly in their world knowledge. This aspect covers how connecting to external data sources and services can enrich the model's responses, making them more grounded in reality and up-to-date.

3. **Structured Reasoning Techniques**: We will delve into structured reasoning as a method to enhance the logical and analytical capabilities of language models. This involves teaching them to process information in a more organized and systematic manner, akin to how a skilled problem solver would approach a complex issue.

Throughout this chapter, we will not only discuss these methods theoretically but also bring them to life through practical applications. Here's a sneak peek into what we'll cover:

- **Combating Hallucinated Content**: A critical shortcoming of current language models is their tendency to produce hallucinated or inaccurate content. We will tackle this issue head-on by introducing automated fact-checking mechanisms. By cross-referencing the model's claims with available evidence, we aim to curb the spread of misinformation, ensuring that the information provided is not only fluent but also factually correct.

- **Mastering Summarization**: A notable strength of language models lies in their ability to summarize content. We will investigate this capability further, examining how to enhance it through varying levels of prompt sophistication. Particularly for lengthy documents, we will introduce the concept of the map-reduce approach, a technique borrowed from computer science that helps in managing and summarizing extensive information efficiently.

- **Extracting Information with Precision**: Moving forward, we will discuss how function calls can be used for extracting specific information from documents. This is a step towards more targeted and purposeful interactions with language models, where the focus is on retrieving precise data points rather than just general information.

- **Application Development with Tool Integration**: To demonstrate the power of tool integration, we will develop an application that exemplifies how connecting to external data and services can greatly enhance the language model's utility, especially in compensating for its limited knowledge of the world.

- **Applying Reasoning Strategies**: Finally, we will push the boundaries further by incorporating advanced reasoning strategies into our application. This will showcase how a language model, when equipped with the right tools and techniques, can not only process information but also reason through it in a more human-like manner.

In summary, this chapter aims to transform the impressive fluency of language models into practical, reliable, and intelligent assistance. By bridging the gap between raw linguistic ability and applied intelligence, we're stepping into a future where language models become indispensable tools in our daily lives.


In [2]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI
from config_api import set_environment
set_environment()

In [4]:
llm = OpenAI(temperature=0.7, model="gpt-3.5-turbo-instruct")
text = "What type of mammal lays the biggest eggs?"
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)
checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


' The question is contradictory as mammals do not lay eggs. The largest eggs are laid by birds, such as the ostrich or emu, which are not mammals.'

In [5]:
text = """Monotremes, a type of mammal found in Australia and parts of New Guinea, lay the largest eggs in the mammalian world. The eggs of the American echidna (spiny anteater) can grow as large as 10 cm in length, and dunnarts (mouse-sized marsupials found in Australia) can have eggs that exceed 5 cm in length.
• Monotremes can be found in Australia and New Guinea
• The largest eggs in the mammalian world are laid by monotremes
• The American echidna lays eggs that can grow to 10 cm in length
• Dunnarts lay eggs that can exceed 5 cm in length
• Monotremes can be found in Australia and New Guinea – True
• The largest eggs in the mammalian world are laid by monotremes – True
• The American echidna lays eggs that can grow to 10 cm in length – False, the American echidna lays eggs that are usually between 1 to 4 cm in length.
• Dunnarts lay eggs that can exceed 5 cm in length – False, dunnarts lay eggs that are typically between 2 to 3 cm in length.
The largest eggs in the mammalian world are laid by monotremes, which can be found in Australia and New Guinea. Monotreme eggs can grow to 10 cm in length.
"""

In [6]:
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)
checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


' True'

In [8]:
from langchain import OpenAI
prompt = """
Summarize this text in one sentence:
{text}
"""
llm = OpenAI(model="gpt-3.5-turbo-instruct")
summary = llm(prompt.format(text=text))

In [11]:
summary

'The speaker wants to share a dull story from their childhood.'

In [10]:
from langchain_decorators import llm_prompt
@llm_prompt
def summarize(text:str, length="short") -> str:
    """
    Summarize this text in {length} length:
    {text}
    """
    return
summary = summarize(text="let me tell you a boring story from when I was young...")

In [13]:
from langchain import PromptTemplate, OpenAI
from langchain.schema import StrOutputParser
llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template(
    "Summarize this text: {text}?"
)
runnable = prompt | llm | StrOutputParser()
summary = runnable.invoke({"text": text})

In [14]:
summary

"\n\nMonotremes, found in Australia and parts of New Guinea, lay the largest eggs in the mammalian world. However, the eggs of the American echidna and dunnarts, both types of monotremes, are not as large as previously thought. While the American echidna's eggs can reach up to 10 cm in length, they are typically between 1 to 4 cm long. Dunnarts, on the other hand, lay eggs that are usually between 2 to 3 cm in length, not exceeding 5 cm. "

In [15]:
template = """Article: { text }
You will generate increasingly concise, entity-dense summaries of the above article.
Repeat the following 2 steps 5 times.
Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.
A missing entity is:
- relevant to the main story,
- specific yet concise (5 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).
Guidelines:
- The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
"""

In [16]:
from langchain import PromptTemplate, OpenAI
from langchain.schema import StrOutputParser
llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template(
    "Summarize this text: {text}?"
)
runnable = prompt | llm | StrOutputParser()
summary = runnable.invoke({"text": text})

In [17]:
summary 

'\nMonotremes, found in Australia and New Guinea, lay the largest mammalian eggs, with the American echidna and dunnarts having eggs that typically range from 1 to 4 cm and 2 to 3 cm in length, respectively.'

In [21]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI
from langchain.document_loaders import PyPDFLoader
pdf_file_path = "matplotlib.pdf"
pdf_loader = PyPDFLoader(pdf_file_path)
docs = pdf_loader.load_and_split()
llm = OpenAI(model="gpt-3.5-turbo-instruct")
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

' \nThe article discusses the benefits and challenges of using Matplotlib for data visualization in Python, particularly for business analysis. It includes tips for effectively using Matplotlib, such as learning its terminology and using the object-oriented interface. The article also provides links to helpful resources and code examples. There is also a discussion on the effectiveness of Matplotlib in the business world and comparisons to other data visualization tools.'

In [23]:
from langchain import OpenAI, PromptTemplate
from langchain.callbacks import get_openai_callback
llm_chain = PromptTemplate.from_template("Tell me a joke about {topic}!") | OpenAI(model="gpt-3.5-turbo-instruct")
with get_openai_callback() as cb:
    response = llm_chain.invoke(dict(topic="light bulbs"))
    print(response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")


Why did the light bulb go to therapy?
Because it was feeling a bit burned out.
Total Tokens: 27
Prompt Tokens: 8
Completion Tokens: 19
Total Cost (USD): $0.0


In [24]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]
llm_chain.generate(input_list)

AttributeError: 'RunnableSequence' object has no attribute 'generate'